In [ ]:
#@title Aprendizaje Profundo | Otoño 2021 by Datitos{display-mode: "form" }
#@markdown ![71335171.png](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAMgAAADICAIAAAAiOjnJAAACwElEQVR4nOzdMY7iQBBA0WU197/FnJNNJ/FqWvLHZfd7McIGfVVQos3X+/3+A2f7e/UN8EzCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBJfV9/A/7xer6XX3/1/gZ70eU0sEsIiISwSwiIhLBLCIiEsEiP2WEf7m9U9zVnvU9vh85pYJIRFQlgkhEVCWCSERUJYJEbssc5ytL+5at8zec9UM7FICIuEsEgIi4SwSAiLhLBIPGqPdWR1v1VfdwcmFglhkRAWCWGREBYJYZEQFokt9lhHdt4z1UwsEsIiISwSwiIhLBLCIiEsEh/dY+18zq4w7RzlTyYWCWGREBYJYZEQFglhkRAWiS1+jzVhr/Mbd7nP3zCxSAiLhLBICIuEsEgIi4SwSDxqj1U/72r1ulc9l2sCE4uEsEgIi4SwSAiLhLBICIvE7D3W9/fSy6/63dLqdZfvc/F7mMDEIiEsEsIiISwSwiIhLBLCIjF7j3WSs87rTXufyUwsEsIiISwSwiIhLBLCIiEsEq8Ju5N6r3OXc3xP2oeZWCSERUJYJIRFQlgkhEVCWCRG7LGOTN7TfNIdvwcTi4SwSAiLhLBICIuEsEgIi8Toc4U7Pyf9p8n7qiMmFglhkRAWCWGREBYJYZEQFonRe6wjd9zr7MbEIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIiISwSwiIhLBLCIiEsEsIi8S8AAP//HtRtH09JwIEAAAAASUVORK5CYII=)
#El siguiente notebook fue traducido por Pablo Marinozi como el séptimo trabajo práctico correspondiente a la versión de Otoño del 2021 del curso Aprendizaje Profundo organizado por Datitos
#El tutorial original fue diseñado por Ben Trevett y fue publicado en su github https://github.com/bentrevett
#Para mayor información consultar https://datitos.github.io/curso-aprendizaje-profundo/#calendario

#Trabajo Práctico N° 7: Aprendizaje Secuencia a Secuencia con Redes Neuronales

# Sección 1 - Encoder-Decoder con RNNs

En este práctico, crearemos un modelo de aprendizaje automático para pasar de una secuencia a otra, utilizando PyTorch y torchtext. Esto se hará en traducciones del alemán al inglés, pero los modelos se pueden aplicar a cualquier problema que implique pasar de una secuencia a otra, como el resumen, es decir, pasar de una secuencia a una secuencia más corta en el mismo idioma.

En este primer notebook, comenzaremos a comprender los conceptos generales de manera simple mediante la implementación del modelo del documento [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215).

## Introducción

Los modelos de secuencia a secuencia (seq2seq) más comunes son los modelos *encoder-decoder*, que comúnmente usan una *red neuronal recurrente* (RNN) para *codificar* la oración de origen (entrada) en un solo vector. En este notebook, nos referiremos a este vector único como *vector de contexto*. Podemos pensar en el vector de contexto como una representación abstracta de toda la oración de entrada. Este vector es luego *decodificado* por un segundo RNN que aprende a generar la oración de destino (de salida) generándola una palabra a la vez.

![](https://i.imgur.com/HZPIbyC.png)

La imagen de arriba muestra un ejemplo de traducción. La oración de entrada/fuente, "guten morgen", se pasa a través de la capa de embedding (amarillo) y luego se ingresa en el encoder (verde). También agregamos un símbolo de *inicio de secuencia* (`<sos>`) y *fin de secuencia* (`<eos>`) al inicio y al final de la oración, respectivamente. En cada paso de tiempo, la entrada al encoder RNN es tanto el embedding, $e$, de la palabra actual, $e(x_t)$, como el estado oculto del paso de tiempo anterior, $ h_{t -1} $, y el codificador RNN genera un nuevo estado oculto $ h_t $. Podemos pensar en el estado oculto como una representación vectorial de la oración hasta ahora. El RNN se puede representar como una función de $ e(x_t) $ y $ h_{t-1} $:

$$h_t = \text{EncoderRNN}(e(x_t), h_{t-1})$$

Usamos el término RNN generalmente aquí, podría ser cualquier arquitectura recurrente, como una *LSTM* (Long Short-Term Memory) o una *GRU* (Gated Recurrent Unit).

Aquí, tenemos $ X = \{x_1, x_2, ..., x_T \} $, donde $ x_1 = \text{<sos>}, x_2 = \text{guten} $, etc. El estado oculto inicial, $ h_0 $, generalmente se inicializa en ceros o en un parámetro aprendido.

Una vez que la última palabra, $ x_T $, se ha pasado al RNN a través de la capa de embedding, usamos el estado oculto final, $ h_T $, como vector de contexto, es decir, $ h_T = z $. Esta es una representación vectorial de toda la oración fuente.

Ahora que tenemos nuestro vector de contexto, $ z $, podemos comenzar a decodificarlo para obtener la oración de salida/objetivo, "good morning". Nuevamente, agregamos tokens de inicio y final de secuencia a la oración de destino. En cada paso de tiempo, la entrada al decoder RNN (azul) es el embedding, $ d $, de la palabra actual, $ d(y_t) $, así como el estado oculto del paso de tiempo anterior, $ s_{ t-1} $, donde el estado oculto inicial del decoder , $ s_0 $, es el vector de contexto, $ s_0 = z = h_T $, es decir, el estado oculto inicial del decoder es el estado oculto final del encoder. Por lo tanto, de manera similar al encoder, podemos representar el decoder como:

$$s_t = \text{DecoderRNN}(d(y_t), s_{t-1})$$

Aunque la capa de embedding de entrada/origen, $ e $, y la capa de embedding de salida/destino, $ d $, se muestran en amarillo en el diagrama, son dos capas de embedding diferentes con sus propios parámetros.

En el decoder, necesitamos pasar del estado oculto a una palabra real, por lo tanto, en cada paso de tiempo usamos $ s_t $ para predecir (pasándolo a través de una capa `Densa`, que se muestra en violeta) la que pensamos que es la siguiente palabra en la secuencia, $ \hat{y}_t $.

$$\hat{y}_t = f(s_t)$$

Las palabras en el decoder siempre se generan una tras otra, con una por paso de tiempo. Siempre usamos `<sos>` para la primera entrada al decoder, $ y_1 $, pero para las entradas subsiguientes, $ y_{t> 1} $, a veces usaremos la siguiente palabra de la secuencia ground truth, $ y_t $ y a veces usamos la palabra predicha por nuestro decoder, $ \hat{y}_{t-1} $. Esto se llama *forzamiento del maestro*, vea un poco más de información al respecto [aquí](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/).

Al entrenar/evaluar nuestro modelo, siempre sabemos cuántas palabras hay en nuestra oración objetivo, por lo que dejamos de generar palabras una vez que alcanzamos esa cantidad. Durante la inferencia, es común seguir generando palabras hasta que el modelo genera un token `<eos>` o después de que se haya generado una cierta cantidad de palabras.

Una vez que tenemos nuestra oración objetivo predicha, $ \hat{Y} = \{\hat{y} _1, \hat{y} _2, ..., \hat{y}_T \} $, la comparamos con nuestra oración objetivo real, $ Y = \{y_1, y_2, ..., y_T \} $, para calcular nuestra pérdida. Luego usamos esta pérdida para actualizar todos los parámetros en nuestro modelo.



## Preparando los datos

Vamos a programar los modelos en PyTorch y usar torchtext para ayudarnos a hacer todo el preprocesamiento requerido. También usaremos spaCy para ayudar en la tokenización de los datos.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator, Example, Dataset

import spacy
import numpy as np

import random
import math
import time

Vamos a establecer las semillas aleatorias para obtener resultados deterministas.

In [2]:
SEED = 1234
  
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

A continuación, vamos a crear los tokenizadores. Un tokenizador se usa para convertir una cadena que contiene una oración en una lista de tokens individuales que componen esa cadena, p. Ej. "Good morning!" se convierte en ["good", "morning", "!"]. Empezaremos a tratar las oraciones como una secuencia de tokens a partir de ahora, en lugar de decir que son una secuencia de palabras. 

spaCy tiene un modelo para cada idioma ("de_core_news_sm" para alemán y "en_core_web_sm" para inglés) que debe cargarse para poder acceder al tokenizador de cada modelo.

**Nota**: descarga los modelos de spaCy ejecutando la siguiente celda




In [3]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9MB 15.5MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=0563c0d1ed6c4ab9aa807d1d001a4b03e8dd7e3e2b4cbbbeea96d52db42a56ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-ze1ntkuf/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


Después de descargar los modelos, tenemos que reiniciar el entorno de ejecución y correr la siguiente celda.

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

A continuación, vamos a crear las funciones del tokenizador. Estas se pueden pasar a torchtext y transformarán la oración en una lista de tokens.

En el paper que estamos implementando, les resulta beneficioso invertir el orden de la entrada porque creen que "introduce muchas dependencias a corto plazo en los datos que facilitan mucho el problema de optimización". Copiamos esto invirtiendo la oración en alemán después de que se haya transformado en una lista de tokens.


In [4]:
def tokenize_de(text):
    """
    Tokeniza el texto en alemán de una cadena a una lista de cadenas (tokens) y lo invierte
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokeniza el texto en inglés de una cadena a una lista de cadenas (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Los `Field`s de torchtext manejan cómo se deben procesar los datos. Todos los argumentos posibles se detallan [aquí](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L61).

Seteamos el argumento `tokenize` en la función de tokenización correcta para cada uno, siendo el alemán el Field ` SRC` (origen) y el inglés el Field `TRG` (destino). El Field también agrega los tokens de "inicio de secuencia" y "fin de secuencia" a través de los argumentos `init_token` y` eos_token`, y convierte todas las palabras a minúsculas.

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

A continuación, descargamos y cargamos los datasets de entrenamiento, validación y prueba.

El dataset que usaremos es el [Multi30k](https://github.com/multi30k/dataset). Este es un conjunto de datos con ~30,000 oraciones paralelas en inglés, alemán y francés, cada una con ~12 palabras por oración.

`exts` especifica qué idiomas usar como origen y destino (el origen va primero) y ` fields` especifica qué Fields usar para cada uno.

In [10]:
#train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
 #                                                   fields = (SRC, TRG))

Esta etapa de tokenización puede tardar alrededor de 20 minutos en ejecutarse. Para evitar tener que esperar tanto tiempo cada vez que generemos una sesión en Colab, vamos a guardar el dataset tokenizado en archivos.

**Nota: Los archivos se borrarán cuando se desconecte la sesión. Asegurate de descargarlos a tu máquina o guardarlos en tu drive.** 

In [7]:
import json
def save_examples(dataset, savepath):
    with open(savepath, 'w') as f:
        # Save num. elements (not really need it)
        f.write(json.dumps(len(dataset.examples)))  # Write examples length
        f.write("\n")

        # Save elements
        for pair in dataset.examples:
            data = [pair.src, pair.trg]
            f.write(json.dumps(data))  # Write samples
            f.write("\n")


def load_examples(filename):
    examples = []
    with open(filename, 'r') as f:
        # Read num. elements (not really need it)
        total = json.loads(f.readline())

        # Save elements
        for i in range(total):
            line = f.readline()
            example = json.loads(line)
            # example = data.Example().fromlist(example, fields)  # Create Example obj. (you can do it here or later)
            examples.append(example)
    return examples

In [8]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DatitosTP2/
%ls

Mounted at /content/drive
/content/drive/MyDrive/DatitosTP2
fifa2021_test.csv      test.json   tut1-model.pt  valid.json
fifa2021_training.csv  train.json  tut2-model.pt


In [ ]:
# #Guarda los datasets tokenizados en disco
# save_examples(train_data,"/content/drive/MyDrive/DatitosTP2/train.json")
# save_examples(valid_data,"/content/drive/MyDrive/DatitosTP2/valid.json")
# save_examples(test_data,"/content/drive/MyDrive/DatitosTP2/test.json")

In [9]:
#Carga los datasets tokenizados desde los archivos
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
fields = [('src', SRC), ('trg', TRG)]

examples = load_examples("/content/drive/MyDrive/DatitosTP2/train.json")
examples = [Example().fromlist(d, fields) for d in examples]
train_data = Dataset(examples, fields)

examples = load_examples("/content/drive/MyDrive/DatitosTP2/valid.json")
examples = [Example().fromlist(d, fields) for d in examples]
valid_data = Dataset(examples, fields)

examples = load_examples("/content/drive/MyDrive/DatitosTP2/test.json")
examples = [Example().fromlist(d, fields) for d in examples]
test_data = Dataset(examples, fields)

Podemos verificar que hayamos cargado la cantidad correcta de ejemplos:

In [10]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")


Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


También podemos imprimir un ejemplo, asegurándonos de que la oración original esté invertida:

In [ ]:
print(vars(train_data.examples[0]))

El punto está al principio de la oración en alemán, por lo que parece que la oración se ha invertido correctamente.

A continuación, vamos a armar el *vocabulario* para los idiomas de origen y destino. El vocabulario se usa para asociar cada token único con un índice (un número entero). Los vocabularios de los idiomas de origen y de destino son distintos.

Usando el argumento `min_freq`, solo permitimos que los tokens que aparecen al menos 2 veces aparezcan en nuestro vocabulario. Los tokens que aparecen solo una vez se convierten en un token `<unk>` (desconocido).

Es importante tener en cuenta que nuestro vocabulario solo debe construirse a partir del conjunto de entrenamiento y no del conjunto de validación/prueba. Esto evita la "fuga de información" en nuestro modelo, dándonos puntuaciones de validación/prueba artificialmente infladas.

In [11]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

El paso final de la preparación de los datos es crear los iteradores. Estos se pueden iterar para devolver un lote de datos que tendrá un atributo `src` (los tensores PyTorch que contienen un lote de oraciones en el idioma origen numericalizadas) y un atributo` trg` (los tensores PyTorch que contienen un lote de oraciones en el idioma objetivo numericalizadas). "Numericalizada" es solo una forma elegante de decir que se han convertido de una secuencia de tokens legibles a una secuencia de índices correspondientes, usando el vocabulario.

También necesitamos definir un `dispositivo`. Esto se usa para decirle a torchText que ponga los tensores en la GPU o no. Usamos la función `torch.cuda.is_available()`, que devolverá `True` si se detecta una GPU en nuestra computadora. Pasamos este "dispositivo" al iterador.

Cuando obtenemos un lote de ejemplos usando un iterador, debemos asegurarnos de que todas las oraciones fuente se rellenen con la misma longitud, lo mismo con las oraciones objetivo. ¡Afortunadamente, los iteradores de torchText manejan esto por nosotros!

Usamos un `BucketIterator` en lugar del` Iterator` estándar ya que crea lotes de tal manera que minimiza la cantidad de relleno en las oraciones de origen y de destino.

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False, 
    device = device)

## Construyendo el modelo Seq2Seq

Construiremos nuestro modelo en tres partes. El encoder, el decoder y un modelo seq2seq que los encapsula a ambos y proporcionará una forma de interactuar con cada uno.

### Encoder

Primero, el encoder es un LSTM de 2 capas. El paper que estamos implementando utiliza un LSTM de 4 capas, pero en aras de disminuir el tiempo de entrenamiento, lo redujimos a 2 capas. El concepto de RNN multicapa es fácil de expandir de 2 a 4 capas.

Para un RNN multicapa, la oración de entrada, $ X $, después de pasar por una capa de embedding va a la primera capa (la inferior) del RNN y los estados ocultos, $ H = \{h_1, h_2, ..., h_T \} $ , producidos por esta capa se utilizan como entradas para el RNN en la capa superior. Por lo tanto, al representar cada capa con un superíndice, los estados ocultos en la primera capa vienen dados por:

$$h_t^1 = \text{EncoderRNN}^1(e(x_t), h_{t-1}^1)$$

Los estados ocultos en la segunda capa vienen dados por:

$$h_t^2 = \text{EncoderRNN}^2(h_t^1, h_{t-1}^2)$$

El uso de un RNN multicapa también significa que necesitaremos un estado oculto inicial como entrada por capa, $ h_0 ^ l $, y también generaremos un vector de contexto por capa, $ z ^ l $.

Sin entrar en demasiados detalles sobre las LSTM (ver [esto](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) para obtener más información sobre ellas), todo lo que necesitamos saber es que son un tipo de RNN que en lugar de simplemente tomar un estado oculto y devolver un nuevo estado oculto por paso de tiempo, también toma y devuelve un *estado de celda*, $ c_t $, por paso de tiempo.

$$\begin{align*}
h_t &= \text{RNN}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})
\end{align*}$$

Podemos pensar en $ c_t $ como otro tipo de estado oculto. Similar a $ h_0 ^ l $, $ c_0 ^ l $ se inicializará a un tensor de todos ceros. Además, nuestro vector de contexto ahora será tanto el estado oculto final como el estado final de la celda, es decir, $ z ^ l = (h_T ^ l, c_T ^ l) $.

Al extender nuestras ecuaciones multicapa a LSTM, obtenemos:

$$\begin{align*}
(h_t^1, c_t^1) &= \text{EncoderLSTM}^1(e(x_t), (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2) &= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Observe cómo solo nuestro estado oculto de la primera capa se pasa como entrada a la segunda capa, y no el estado de la celda.

Entonces nuestro encoder se parece a esto:

![](https://i.imgur.com/4s0fceB.png)

Creamos esto en código haciendo un módulo `Encoder`, que requiere que heredemos de `torch.nn.Module` y usemos `super().__ init __()` como código repetitivo. El encoder toma los siguientes argumentos:
- `input_dim` es el tamaño/dimensionalidad de los vectores one-hot que se ingresarán al encoder. Esto es igual al tamaño del vocabulario de entrada (origen).
- `emb_dim` es la dimensionalidad de la capa de embedding. Esta capa convierte los vectores one-hot en vectores densos con dimensión `emb_dim`.
- `hid_dim` es la dimensionalidad de los estados ocultos y de celda.
- `n_layers` es el número de capas en el RNN.
- `dropout` es la probabilidad de dropout que se debe utilizar. Este es un parámetro de regularización para evitar el sobreajuste. Consulte [esto](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) para obtener más detalles sobre el dropout.

No vamos a discutir la capa de embedding en detalle durante estos tutoriales. Todo lo que necesitamos saber es que hay un paso antes de que las palabras (técnicamente, los índices de las palabras) se pasen al RNN, donde las palabras se transforman en vectores. Para leer más sobre embedding de palabras, consulte estos artículos: [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/ ), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/).

La capa de embedding se crea usando `nn.Embedding`, la LSTM con `nn.LSTM` y la capa de dropout con `nn.Dropout`. Consulte la [documentación](https://pytorch.org/docs/stable/nn.html) de PyTorch para obtener más información sobre estas.

Una cosa a tener en cuenta es que el argumento "dropout" del LSTM es cuánto dropout aplicar entre las capas de un RNN multicapa, es decir, entre la salida de estados ocultos de la capa $ l $ y esos mismos estados ocultos que se utilizan para la entrada de la capa $ l + 1 $.

En el método `forward`, pasamos la oración de origen, $ X $, que se convierte en vectores densos usando la capa` embedding`, y luego se aplica el dropout. Estos embeddings luego se pasan al RNN. A medida que pasamos una secuencia completa al RNN, ¡automáticamente hará el cálculo recurrente de los estados ocultos en toda la secuencia por nosotros! Tenga en cuenta que no pasamos un estado inicial oculto o de celda al RNN. Esto se debe a que, como se indica en la [documentación](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), si no se pasa ningún estado oculto/de celda al RNN, se crea automáticamente un estado oculto/de celda inicial como un tensor de todos ceros.

El RNN devuelve: `outputs` (el estado oculto de la capa superior para cada paso de tiempo), `hidden` (el estado oculto final de cada capa, $ h_T $, apilados uno encima del otro) y `cell` (el estado de celda final para cada capa, $ c_T $, apilados uno encima del otro).

Como solo necesitamos los estados finales ocultos y de celda (para hacer nuestro vector de contexto), `forward` solo devuelve `hidden` y `cell`.

Los tamaños de cada uno de los tensores se dejan como comentarios en el código. En esta implementación, `n_directions` siempre será 1, sin embargo, tenga en cuenta que las RNN bidireccionales (usada en el TP 6) tendrán` n_directions` igual a 2.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el encoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.lstm = nn.LSTM(self.emb_dim, self.hid_dim,self.n_layers)
        self.dropout_ = nn.Dropout(self.dropout)


        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, src):
        ########################################################################
        # TODO: Implementá la función forward para el encoder. Deberías        #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        
        #src = [src len, batch size]
        x = src

        #embedded = [src len, batch size, emb dim]
        x = self.embedding(x)
        x = self.dropout_(x)

        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        outputs, (hidden, cell) = self.lstm(x)
       

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        return hidden, cell

### Decoder

A continuación, construiremos nuestro decoder, que también será un LSTM de 2 capas (4 en el paper).

![](https://i.imgur.com/aTf2uPT.png)

La clase `Decoder` realiza un solo paso de decodificación, es decir, genera un solo token por paso de tiempo. La primera capa recibirá un estado oculto y de celda del paso de tiempo anterior, $ (s_{t-1} ^ 1, c_{t-1} ^ 1) $, y lo alimenta a través del LSTM con el embedding del token actual , $ y_t $, para producir un nuevo estado oculto y de celda, $ (s_t ^ 1, c_t ^ 1) $. Las capas siguientes utilizarán el estado oculto de la capa inferior, $ s_t ^ {l-1} $, y los estados ocultos y de celda anteriores de su misma capa, $ (s_{t-1} ^ l, c_{t-1 } ^ l) $. Esto proporciona ecuaciones muy similares a las del encoder.

$$\begin{align*}
(s_t^1, c_t^1) = \text{DecoderLSTM}^1(d(y_t), (s_{t-1}^1, c_{t-1}^1))\\
(s_t^2, c_t^2) = \text{DecoderLSTM}^2(s_t^1, (s_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Recuerde que los estados ocultos y de celda iniciales de nuestro decoder son nuestros vectores de contexto, que son los estados ocultos y de celda finales de nuestro encoder de la misma capa, es decir, $ (s_0 ^ l, c_0 ^ l) = z ^ l = (h_T ^ l, c_T ^ l) $.

Luego pasamos el estado oculto de la capa superior del RNN, $ s_t ^ L $, a través de una capa lineal, $ f $, para hacer una predicción de cuál debería ser el siguiente token en la secuencia objetivo (salida), $ \hat{y}_{t + 1} $. 

$$\hat{y}_{t+1} = f(s_t^L)$$

Los argumentos y la inicialización son similares a la clase `Encoder`, excepto que ahora tenemos un `output_dim` que es el tamaño del vocabulario para la salida/destino. También hay una adición de la capa `Linear`, que se utiliza para hacer las predicciones desde el estado oculto de la capa superior.

Dentro del método `forward`, aceptamos un lote de tokens de entrada, estados ocultos anteriores y estados de celda anteriores. Como solo decodificamos un token a la vez, los tokens de entrada siempre tendrán una longitud de secuencia de 1. Hacemos un `unsqueeze` sobre los tokens de entrada para agregar una dimensión de longitud de oración de 1. Luego, de manera similar al encoder, pasamos a través de un capa de embedding y aplicamos dropout. Este lote de embeddings de tokens se pasa luego al RNN con los estados ocultos y de celda anteriores. Esto produce un tensor `output` (estado oculto de la capa superior del RNN), un nuevo tensor de estados ocultos  `hidden`  (uno para cada capa, apilados uno encima del otro) y un nuevo tensor de estados de celda `cell`  (también uno por capa, apilados uno encima del otro). Luego pasamos `output` (después de deshacernos de la dimensión de longitud de la oración) a través de la capa densa para obtener nuestra `prediction`. Luego devolvemos la `prediction`, el nuevo estado oculto `hidden` y el nuevo estado de la celda `cell`.


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el decoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        self.lstm = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers, dropout = self.dropout)
        self.linear = nn.Linear(self.hid_dim, self.output_dim)
        self.dropout_ = nn.Dropout(self.dropout)
        

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, input, hidden, cell):
        ########################################################################
        # TODO: Implementá la función forward para el decoder. Deberías        #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions en el decoder será siempre 1, por lo tanto:
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        x = torch.unsqueeze(input, 0)
        #input = [1, batch size]       

        x = self.embedding(x)
        x = self.dropout_(x)
        #embedded = [1, batch size, emb dim]

        outputs, (hidden, cell) = self.lstm(x, (hidden, cell))
        
        
        outputs = torch.squeeze(outputs,0)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len y n directions en el decoder serán siempre 1, por lo tanto:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.linear(outputs)
        #print(prediction)
        
        #prediction = [batch size, output dim]

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        return prediction, hidden, cell

### Seq2Seq

Para la parte final de la implementación, desarrollaremos el modelo seq2seq. Este modelo se encargará de:
- recibir la frase de entrada/origen
- usar el encoder para producir los vectores de contexto
- usar el decoder para producir la predicción de la frase de salida/destino

Nuestro modelo completo se verá así:

![](https://i.imgur.com/uTI0sgy.png)

El modelo `Seq2Seq` incluye un `Encoder`, un `Decoder` y un `device` (utilizado para colocar tensores en la GPU, si hubiera).

Para esta implementación, tenemos que asegurarnos de que el número de capas y las dimensiones ocultas (y de celda) sean iguales en el `Encoder` y en el `Decoder`. Este no es siempre el caso, no necesariamente tenemos que tener el mismo número de capas o los mismos tamaños de dimensión ocultos en un modelo secuencia a secuencia. Sin embargo, si hiciéramos algo como tener un número diferente de capas, entonces tendríamos que tomar decisiones sobre cómo se maneja esto. Por ejemplo, si nuestro encoder tiene 2 capas y nuestro decoder solo tiene 1, ¿cómo se maneja esto? ¿Promediamos los dos vectores de contexto generados por el encoder? ¿Pasamos ambos por una capa lineal? ¿Usamos solo el vector de contexto de la capa más alta? Etc.

Nuestro método `forward` toma la oración de origen, la oración de destino y un ratio de forzamiento del maestro. Esta última se utiliza al entrenar nuestro modelo. Al decodificar, en cada paso de tiempo predeciremos cuál será el siguiente token en la secuencia objetivo a partir de los tokens decodificados previamente, $ \hat{y}_{t + 1} = f(s_t ^ L) $. Con una probabilidad igual al ratio de forzamiento del maestro (`teacher_forcing_ratio`) usaremos el token de ground truth como entrada al decodificador durante el siguiente paso de tiempo. Sin embargo, con probabilidad `1 - teacher_forcing_ratio`, usaremos el token que el modelo predijo como la próxima entrada al modelo, incluso si no coincide con el siguiente token ground truth.

Lo primero que hacemos en el método `forward` es crear un tensor de `output` que almacenará todas nuestras predicciones, $ \hat{Y} $.

Luego alimentamos la oración de entrada/origen, `src`, en el encoder y recibimos los estados finales ocultos y de celda.

La primera entrada al decoder es el token de inicio de secuencia (`<sos>`). Como nuestro tensor `trg` ya tiene el token` <sos> `adjunto (desde que definimos el `init_token` en nuestro Field `TRG`) obtenemos nuestro $ y_1 $ haciendo un slice. Sabemos qué tan largo deben ser nuestras oraciones de destino (`max_len`), por lo que lo repetimos esa cantidad de veces. El último token de entrada en el decoder es el token **anterior** `<eos>` - el token `<eos>` **nunca** se ingresa en el decoder.

Durante cada iteración del ciclo, nosotros:
- pasamos la entrada, los estados ocultos y de la celda anteriores ($ y_t, s_{t-1}, c_{t-1} $) al decoder
- recibir una predicción, el siguiente estado oculto y el siguiente estado de celda ($ \hat{y}_{t + 1}, s_{t}, c_{t} $) del decoder
- colocar nuestra predicción, $ \hat{y}_{t + 1} $ / `output` en nuestro tensor de predicciones, $ \hat{Y} $ / `outputs`
- decidir si vamos a hacer un forzamiento del maestro o no
     - si lo hacemos, el siguiente `input` es el siguiente token de ground truth en la secuencia, $ y_{t + 1} $  `trg[t]`
     - si no lo hacemos, el siguiente `input` es el siguiente token predicho en la secuencia, $ \hat{y}_{t + 1} $ `top1`, que obtenemos haciendo un `argmax` sobre el tensor de salida
    
Una vez que hemos hecho todas nuestras predicciones, devolvemos nuestro tensor lleno de predicciones, $ \hat{Y} $ / `outputs`.

**Nota**: nuestro bucle de decoder comienza en 1, no en 0. Esto significa que el elemento 0 de nuestro tensor de `outputs` permanece solo en ceros. Así que nuestros `trg` y` outputs` se parecen a:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Más adelante, cuando calculamos la pérdida, cortamos el primer elemento de cada tensor para obtener:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el modelo seq2seq con la  #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Las dimensiones de las variables ocultas del encoder y el decoder deben ser iguales!"
        assert encoder.n_layers == decoder.n_layers, \
            "El encoder y el decoder deben tener el mismo número de capas!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio es la probabilidad con la que se usa el forzamiento del maestro
        #e.g. si teacher_forcing_ratio es 0.75 usamos las etiquetas el 75% del tiempo
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor para almacenar las salidas del decoder 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        
        #el último estado oculto del encoder se usa como estado oculto inicial del decoder
        hidden, cell = self.encoder(src)
        
        #la primera entrada al decoder es el token <sos>
        input = trg[0,:]
        

       

        for t in range(1, trg_len):
            
            ########################################################################
            # TODO: Implementá el bucle de decodificación dentro de la función     #
            # forward para el modelo seq2seq. Deberías usar las capas que definiste#     
            # en __init__ y especificar la conectividad de dichas capas            #
            ########################################################################
            # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
            prediction, hidden, cell = self.decoder(input,hidden, cell)
            
            outputs[t] = prediction
            if np.random.uniform(0,1)<teacher_forcing_ratio:
              input = trg[t]
            else:
              input = torch.argmax(prediction, dim=1)

            

            # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
            ########################################################################
            #                          FINAL DE TU CÓDIGO                          #       
            ########################################################################
        
        return outputs

## Entrenamiento del modelo Seq2Seq

Ahora que tenemos nuestro modelo implementado, podemos comenzar a entrenarlo.

Primero, inicializaremos nuestro modelo. Como se mencionó anteriormente, las dimensiones de entrada y salida están definidas por el tamaño del vocabulario. Las dimensiones de embedding y el dropout pueden ser diferentes para el encoder y el decoder, pero el número de capas y el tamaño de los estados ocultos/de celda deben ser los mismos.

Luego definimos el encoder, decoder y luego nuestro modelo Seq2Seq, que colocamos en el `device`.

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

El siguiente paso es inicializar los pesos de nuestro modelo. En el paper, afirman que inicializan todos los pesos a partir de una distribución uniforme entre -0,08 y +0,08, es decir, $ \mathcal{U}(- 0,08, 0,08) $.

Inicializamos pesos en PyTorch creando una función que "aplicamos" a nuestro modelo. Cuando usemos `apply`, la función` init_weights` será llamada en cada módulo y submódulo dentro de nuestro modelo. Para cada módulo, recorremos todos los parámetros y los muestreamos a partir de una distribución uniforme con `nn.init.uniform_`.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

También definimos una función que calculará el número de parámetros entrenables en el modelo.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

Definimos nuestro optimizador, que usamos para actualizar nuestros parámetros en el ciclo de entrenamiento. Consulte [esta](http://ruder.io/optimizing-gradient-descent/) publicación para obtener información sobre los diferentes optimizadores. Aquí, usaremos Adam.

In [ ]:
optimizer = optim.Adam(model.parameters())

A continuación, definimos nuestra función de pérdida. La función `CrossEntropyLoss` calcula tanto el log softmax como la probabilidad logarítmica negativa de nuestras predicciones.

Nuestra función de pérdida calcula la pérdida promedio por token, sin embargo, al pasar el índice del token `<pad>` como el argumento `ignore_index`, ignoramos la pérdida siempre que el token de destino sea un token de relleno.

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

A continuación, definiremos nuestro ciclo de entrenamiento.

Primero, configuraremos el modelo en "modo de entrenamiento" con `model.train()`. Esto activará el dropout (y la normalización por lotes, que no estamos usando) y luego iterará a través de nuestro iterador de datos.

Como se dijo antes, nuestro bucle de decodificador comienza en 1, no en 0. Esto significa que el elemento 0 de nuestro tensor de `outputs` permanece en ceros. Así que nuestros `trg` y `outputs` se parecen a:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Aquí, cuando calculamos la pérdida, cortamos el primer elemento de cada tensor para obtener:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

En cada iteración:
- obtenemos las oraciones de origen y destino del lote, $ X $ y $ Y $
- ponemos a cero los gradientes calculados a partir del último lote
- introducimos el origen y el destino en el modelo para obtener la predicción, $ \hat{Y} $
- como la función de pérdida solo funciona en entradas 2d con objetivos 1d, necesitamos aplanar cada una de ellas con `.view`
     - cortamos la primera columna de la salida y las oraciones destino como se mencionó anteriormente
- calculamos los gradientes con `loss.backward()`
- recortamos los gradientes para evitar que exploten (un problema común en las RNN)
- actualizamos los parámetros de nuestro modelo haciendo un paso de optimización
- sumamos el valor de la pérdida a un total acumulado

Finalmente, devolvemos la pérdida que se promedia en todos los lotes.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Nuestro ciclo de evaluación es similar a nuestro ciclo de entrenamiento, sin embargo, como no estamos actualizando ningún parámetro, no necesitamos pasar un optimizador o un valor de clip.

Debemos recordar configurar el modelo en modo de evaluación con `model.eval()`. Esto desactivará el dropout (y la normalización de lotes, si se usa).

Usamos el bloque `with torch.no_grad()` para asegurarnos de que no se calculen gradientes dentro del bloque. Esto reduce el consumo de memoria y acelera las cosas.

El loop es similar (sin las actualizaciones de los parámetros), sin embargo, debemos asegurarnos de desactivar el forzamiento del maestro para la evaluación. Esto hará que el modelo solo use sus propias predicciones para hacer más predicciones dentro de una oración, lo que refleja cómo se usaría en  producción.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #desactiva el forzamiento del maestro

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

A continuación, crearemos una función que usaremos para decirnos cuánto tarda una época.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

¡Por fin podemos empezar a entrenar nuestro modelo!

En cada época, comprobaremos si nuestro modelo ha logrado la mejor pérdida de validación hasta el momento. Si es así, actualizaremos nuestra mejor pérdida de validación y guardaremos los parámetros de nuestro modelo (llamado `state_dict` en PyTorch). Luego, cuando vayamos a probar nuestro modelo (con el dataset de test), usaremos los parámetros guardados para lograr la mejor pérdida de validación.

Imprimiremos tanto la pérdida como la perplejidad en cada época. Es más fácil ver un cambio en la perplejidad que un cambio en la pérdida, ya que los números son mucho mayores.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Cargaremos los parámetros (`state_dict`) que le dieron a nuestro modelo la mejor pérdida de validación y lo ejecutaremos en el conjunto de prueba.

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

En la siguiente sección implementaremos un modelo que logra una perplejidad de prueba mejorada, pero solo usa una sola capa en el encoder y el decoder.

# Sección 2 - Reduciendo la Compresión de Información
En esta segunda sección sobre modelos secuencia a secuencia usando PyTorch y TorchText, implementaremos el modelo de [Aprendizaje de representaciones de frases usando RNN Encoder-Decoder para traducción automática estadística](https://arxiv.org/abs/1406.1078). Este modelo logrará una perplejidad de prueba mejorada utilizando solo un RNN de una sola capa tanto en el codificador como en el decodificador.

## Introducción

Recordemos el modelo general de encoder-decoder.

![](https://i.imgur.com/09LRqMD.png)

Usamos nuestro encoder (verde) sobre los embedding de la secuencia de origen (amarillo) para crear un vector de contexto (rojo). Luego usamos ese vector de contexto con el decoder (azul) y una capa lineal (violeta) para generar la oración de destino.

En el modelo anterior, usamos una LSTM multicapa como encoder y decoder.

![](https://i.imgur.com/uTI0sgy.png)

Una desventaja del modelo de la sección anterior es que el decoder está tratando de meter mucha información en los estados ocultos. Durante la decodificación, el estado oculto deberá contener información sobre la totalidad de la secuencia fuente, así como todos los tokens que se han decodificado hasta ahora. Al aliviar parte de esta compresión de información, ¡podemos crear un modelo mejor!

También usaremos una GRU (Gated Recurrent Unit) en lugar de una LSTM (Long Short-Term Memory). ¿Por qué? Principalmente porque eso es lo que hicieron en el papaer (este paper también introdujo GRU) y también porque usamos LSTM en la sección anterior. Para comprender en qué se diferencian los GRU (y los LSTM) del RNNS estándar, consulte [este enlace](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). ¿Es un GRU mejor que un LSTM? [La investigación](https://arxiv.org/abs/1412.3555) ha demostrado que son prácticamente iguales, y ambos son mejores que los RNN estándar. 


In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False, 
    device = device)

## Construyendo el modelo Seq2Seq


### Encoder

El codificador es similar al anterior, con el LSTM multicapa intercambiado por un GRU de una sola capa. Tampoco pasamos el dropout como un argumento al GRU ya que ese dropout se usa entre cada capa de un RNN multicapa. Como solo tenemos una capa, PyTorch mostrará una advertencia si intentamos pasarle un valor de dropout.

Otra cosa a tener en cuenta sobre las GRU es que solo requieren y devuelven un estado oculto, no hay un estado de celda como en el LSTM.

$$\begin{align*}
h_t &= \text{GRU}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(e(x_t), h_{t-1})
\end{align*}$$

De las ecuaciones anteriores, parece que el RNN y el GRU son idénticos. Dentro del GRU, sin embargo, hay una serie de *mecanismos de compuertas* que controlan el flujo de información hacia y desde el estado oculto (similar a un LSTM). Nuevamente, para obtener más información, consulte [esta excelente publicación]https://colah.github.io/posts/2015-08-Understanding-LSTMs/).

El resto del encoder debería ser muy familiar desde la sección anterior, toma como entrada una secuencia, $ X = \{x_1, x_2, ..., x_T \} $, la pasa a través de la capa de embedding, calcula de forma recurrente estados ocultos, $ H = \{h_1, h_2, ..., h_T \} $, y devuelve un vector de contexto (el estado oculto final), $ z = h_T $.

$$h_t = \text{EncoderGRU}(e(x_t), h_{t-1})$$

Esto es idéntico al encoder del modelo general seq2seq, con toda la "magia"  ocurriendo dentro del GRU (verde).

![](https://i.imgur.com/Ben5gcM.png)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el encoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.gru= nn.GRU(self.emb_dim, self.hid_dim)
        self.dropout_ = nn.Dropout(self.dropout)

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, src):
        ########################################################################
        # TODO: Implementá la función forward para el encoder. Deberías        #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        

        #src = [src len, batch size]
        
        x = src
        
        #embedded = [src len, batch size, emb dim]
        
        x = self.embedding(x)
        x = self.dropout_(x)

        outputs, hidden = self.gru(x)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #los outputs provienen siempre de la capa oculta superior
        
        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################

        return hidden

### Decoder

El decoder es donde la implementación difiere significativamente del modelo anterior y aliviamos algo de la compresión de información.

En lugar de que el GRU en el decoder tome solo el embedding del token de destino, $ d(y_t) $ y el estado oculto anterior $ s_ t-1} $ como entradas, también toma el vector de contexto $ z $.

$$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

Observe cómo este vector de contexto, $ z $, no tiene un subíndice $ t $, lo que significa que reutilizamos el mismo vector de contexto devuelto por el encoder para cada paso de tiempo en el decoder.

Antes, predijimos el siguiente token, $ \hat{y}_{t + 1} $, con la capa lineal, $ f $, solo usando el estado oculto del decoder de la capa superior en ese paso de tiempo, $ s_t $, como $ \hat{y}_{t + 1} = f(s_t ^ L) $. Ahora, también pasamos el embedding del token actual, $ d(y_t) $ y el vector de contexto, $ z $ a la capa lineal.

$$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

Por lo tanto, nuestro decodificador ahora se parece a esto:

![](https://i.imgur.com/V72jlPu.png)

Tenga en cuenta que el estado oculto inicial, $ s_0 $, sigue siendo el vector de contexto, $ z $, por lo que al generar el primer token, en realidad estamos ingresando dos vectores de contexto idénticos en el GRU.

¿Cómo estos dos cambios reducen la compresión de la información? Bueno, hipotéticamente, los estados ocultos del decoder, $ s_t $, ya no necesitan contener información sobre la secuencia de origen, ya que siempre está disponible como entrada. Por lo tanto, solo necesita contener información sobre los tokens que ha generado hasta ahora. La adición de $ y_t $ a la capa lineal también significa que esta capa puede ver directamente cuál es el token, sin tener que obtener esta información del estado oculto. 

Sin embargo, esta hipótesis es solo una hipótesis, es imposible determinar cómo el modelo realmente usa la información que se le proporciona (no escuche a nadie que diga lo contrario). Sin embargo, es una intuición sólida y los resultados parecen indicar que estas modificaciones son una buena idea.

Dentro de la implementación, pasaremos $ d (y_t) $ y $ z $ al GRU al concatenarlos juntos, por lo que las dimensiones de entrada al GRU ahora son `emb_dim + hid_dim` (ya que el vector de contexto será de tamaño ` hid_dim` ). La capa lineal tomará $ d (y_t), s_t $ y $ z $ también al concatenarlos juntos, por lo tanto, las dimensiones de entrada ahora son `emb_dim + hid_dim * 2`. Tampoco pasamos un valor de dropout al GRU, ya que solo usa una capa.

`forward` ahora toma un argumento `context`. Dentro de `forward`, concatenamos $ y_t $ y $ z $ como` emb_con` antes de alimentar al GRU, y concatenamos $ d (y_t) $, $ s_t $ y $ z $ juntos como `output` antes de alimentarlo a través de la capa lineal para recibir nuestras predicciones, $ \hat{y} _ {t + 1} $.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el decoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        self.gru = nn.GRU(self.emb_dim + self.hid_dim, self.hid_dim)
        self.linear = nn.Linear(self.emb_dim + self.hid_dim*2, self.output_dim)
        self.dropout_ = nn.Dropout(self.dropout)

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################

        
    def forward(self, input, hidden, context):
        ########################################################################
        # TODO: Implementá la función forward para el decoder. Deberías        #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers y n directions en el decoder valdrán ambos siempre 1, por lo tanto:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = torch.unsqueeze(input, 0)
        
        
        #input = [1, batch size]
        
        x = self.embedding(input)
        x = self.dropout_(x)
        
        #embedded = [1, batch size, emb dim]

                
        emb_con = torch.cat((x,context),2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden =self.gru(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions valdrán siempre 1, por lo tanto:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.squeeze(output,0)
        context = torch.squeeze(context,0)
        x = torch.squeeze(x,0)
        # print(context.shape)
        # print(x.shape)
        # print(output.shape)
        output = torch.cat((x,context,output ),1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.linear(output)
        
        #prediction = [batch size, output dim]

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        return prediction, hidden

### Modelo Seq2Seq

Poniendo el encoder y el decoder juntos, obtenemos:

![](https://i.imgur.com/uzewKDu.png)

Nuevamente, en esta implementación debemos asegurarnos de que las dimensiones ocultas tanto en el encoder como en el decoder sean las mismas.

Repasando brevemente todos los pasos:
- el tensor de `outputs` se crea para contener todas las predicciones, $ \hat {Y} $
- la secuencia de origen, $ X $, se alimenta al encoder para recibir un vector de contexto `context`
- el estado oculto del decoder inicial se establece en el vector `context`, $ s_0 = z = h_T $
- usamos un lote de tokens `<sos>` como la primera `entrada`, $ y_1 $
- luego decodificamos dentro de un bucle:
   - insertando el token de entrada $ y_t $, estado oculto anterior, $ s_ {t-1} $, y el vector de contexto, $ z $, en el decoder
   - recibiendo una predicción, $\hat{y}_{t + 1}$, y un nuevo estado oculto, $ s_t$
   - luego decidimos si vamos a hacer forzamiento del maestro o no, estableciendo la siguiente entrada según corresponda (ya sea el siguiente token de ground truth en la secuencia objetivo o el próximo token predicho más alto)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el modelo seq2seq con la  #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Las dimensiones de las variables ocultas del encoder y el decoder deben ser iguales!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio es la probabilidad con la que se usa el forzamiento del maestro
        #e.g. si teacher_forcing_ratio es 0.75 usamos las etiquetas el 75% del tiempo
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor para almacenar las salidas del decoder 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #el último estado oculto del encoder se usa como estado oculto inicial del decoder
        context = self.encoder(src)
        
        #el contexto también es usado como el estado oculto inicial del decoder
        hidden = context
        
        #la primera entrada al decoder es el token <sos>
        input = trg[0,:]
        
        for t in range(1, trg_len):
            ########################################################################
            # TODO: Implementá el bucle de decodificación dentro de la función     #
            # forward para el modelo seq2seq. Deberías usar las capas que definiste#     
            # en __init__ y especificar la conectividad de dichas capas            #
            ########################################################################
            # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
                
            prediction, hidden = self.decoder(input,hidden, context)
            
            outputs[t] = prediction
            if np.random.uniform(0,1)<teacher_forcing_ratio:
              input = trg[t]
            else:
              input = torch.argmax(prediction, dim=1)


            # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
            ########################################################################
            #                          FINAL DE TU CÓDIGO                          #       
            ########################################################################

        return outputs

## Entrenamiento del modelo Seq2Seq

El resto de esta sección es muy similar a la anterior.

Inicializamos nuestro encoder, decoder y modelo seq2seq (colocándolo en la GPU si tenemos una). Como antes, las dimensiones de embedding y la cantidad de dropout utilizada pueden ser diferentes entre el encoder y el decoder, pero las dimensiones ocultas deben seguir siendo las mismas.

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

A continuación, inicializamos nuestros parámetros. El documento establece que los parámetros se inicializan a partir de una distribución normal con una media de 0 y una desviación estándar de 0.01, es decir, $ \mathcal{N}(0, 0.01) $.

También establece que debemos inicializar los parámetros recurrentes a una inicialización especial, sin embargo, para mantener las cosas simples, también los inicializaremos en $ \mathcal{N}(0, 0.01) $.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Imprimimos el número de parámetros.

Aunque solo tenemos un RNN de una sola capa para nuestro encoder y decoder, en realidad tenemos **más** parámetros que el último modelo. Esto se debe al mayor tamaño de las entradas al GRU y la capa lineal. Sin embargo, no es una cantidad significativa de parámetros y provoca un aumento mínimo en el tiempo de entrenamiento (~ 3 segundos extras por época).

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

Inicializamos el optimizador

In [ ]:
optimizer = optim.Adam(model.parameters())

También inicializamos la función de pérdida, asegurándonos de ignorar la pérdida en los tokens `<pad>`.

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Luego creamos el ciclo de entrenamiento ...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

... y el ciclo de evaluación, recordando configurar el modelo en modo `eval` y desactivar el forzamiento del maestro.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #desactiva el forzamiento del maestro

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

También definiremos la función que calcula cuánto tarda una época.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Luego, entrenamos nuestro modelo, guardando los parámetros que nos den la mejor pérdida de validación.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Finalmente, probamos el modelo en el conjunto de prueba usando estos "mejores" parámetros.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Con solo mirar la pérdida de prueba, obtenemos un mejor rendimiento que el modelo anterior. ¡Esta es una muy buena señal de que esta arquitectura modelo está haciendo algo bien! Aliviar la compresión de la información parece la forma de hacerlo, y en el próximo tutorial ampliaremos esto aún más con *atención*.

# Sección 3 - Metiendo Atención

En esta tercer sección sobre modelos de secuencia a secuencia usando PyTorch y TorchText, implementaremos el modelo de [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). Este modelo alcanza nuestra mayor perplejidad hasta el momento, ~ 27 en comparación con ~ 34 para el modelo anterior.


## Introducción

Como recordatorio, aquí está el modelo general de encoder-decoder:

![](https://i.imgur.com/09LRqMD.png)

En la sección anterior, nuestra arquitectura se configuró para reducir la "compresión de información" pasando explícitamente el vector de contexto, $ z $, al decoder en cada paso de tiempo y pasando tanto el vector de contexto como el embedding de la palabra de entrada. , $ d (y_t) $, junto con el estado oculto, $ s_t $, a la capa lineal, $ f $, para hacer una predicción.

![](https://i.imgur.com/3k7EnTX.png)

Aunque hemos reducido parte de esta compresión, nuestro vector de contexto aún necesita contener toda la información sobre la oración fuente. El modelo implementado en esta sección evita esta compresión al permitir que el decodificador vea la oración origen completa (a través de sus estados ocultos) en cada paso de decodificación. ¿Como hace esto? Utiliza *atención*.

La atención funciona primero, calculando un vector de atención, $a$, que tiene la misma longitud de la oración origen. El vector de atención tiene la propiedad de que cada elemento está entre 0 y 1, y el vector completo suma 1. Luego calculamos una suma ponderada de los estados ocultos de nuestra oración origen, $ H $, para obtener un vector fuente ponderado, $ w $ .

$$w = \sum_{i}a_ih_i$$

Calculamos un nuevo vector origen ponderado en cada paso de tiempo al decodificar, usándolo como entrada para nuestro decoder RNN, así como la capa lineal para hacer una predicción. Explicaremos cómo hacer todo esto durante la sección.


## Construyendo el modelo Seq2Seq

### Encoder

Primero, construiremos el encoder. Al igual que en el modelo anterior, solo usamos un GRU de una sola capa, sin embargo ahora usamos un *RNN bidireccional*. Con un RNN bidireccional, tenemos dos RNN en cada capa. Un *RNN hacia adelante* que repasa los embedding de la oración de izquierda a derecha (que se muestra a continuación en verde), y un *RNN hacia atrás* que repasa los embedding de la oración de derecha a izquierda (verde azulado). Todo lo que necesitamos hacer en el código es establecer `bidirectional = True` y luego pasar los embedding de la oración al RNN como antes.

![](https://i.imgur.com/bdUnvj9.png)

Ahora tenemos:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Donde $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

Como antes, solo pasamos una entrada(`embedded`) al RNN, que le dice a PyTorch que inicialice los estados ocultos iniciales hacia adelante y hacia atrás ($ h_0 ^ \rightarrow $ y $ h_0 ^ \leftarrow $, respectivamente) como un tensor de todos los ceros. También obtendremos dos vectores de contexto, uno del RNN hacia adelante después de haber visto la última palabra en la oración, $ z ^ \rightarrow = h_T ^ \rightarrow $, y uno del RNN hacia atrás después de haber visto la primera palabra en la oración, $ z ^ \leftarrow = h_T ^ \leftarrow $.

El RNN devuelve `outputs` y `hidden`.

`outputs` es de tamaño **[longitud de la oración de origen, tamaño de lote, (dimensión de variables ocultas)*(numero de direcciones)] ** donde los primeros ` hid_dim` elementos en el tercer eje son los estados ocultos de la capa superior hacia adelante RNN, y los últimos `hid_dim ` elementos son estados ocultos de la capa superior hacia atrás RNN. Podemos pensar en el tercer eje como los estados ocultos hacia adelante y hacia atrás concatenados entre sí, es decir, $ h_1 = [h_1 ^ \rightarrow; h_{T} ^ \leftarrow] $, $ h_2 = [h_2 ^ \rightarrow; h_{T-1} ^ \leftarrow] $ y podemos denotar todos los estados ocultos del encoder (hacia adelante y hacia atrás concatenados juntos) como $ H = \{h_1, h_2, ..., h_T \} $. 

`hidden` es de tamaño **[número de capas * número de direcciones, tamaño de lote, dimensión de variables ocultas]**, donde **[- 2,:,:]** entrega el estado oculto de la capa superior de la RNN hacia adelante después del paso de tiempo final (es decir, después de haber visto la última palabra en la oración) y ** [- 1,:,: ]** entrega el estado oculto de la capa superior de la RNN hacia atrás después del paso de tiempo final (es decir, después de haber visto la primera palabra en la oración).

Como el decoder no es bidireccional, solo necesita un único vector de contexto, $ z $, para usar como su estado oculto inicial, $ s_0 $, y actualmente tenemos dos, uno hacia adelante y uno hacia atrás ($ z ^ \rightarrow = h_T ^ \rightarrow $ y $ z ^ \leftarrow = h_T ^ \leftarrow $, respectivamente). Resolvemos esto concatenando los dos vectores de contexto juntos, pasándolos a través de una capa lineal, $ g $, y aplicando la función de activación $ \tanh $. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

Como queremos que nuestro modelo mire hacia atrás sobre la totalidad de la oración origen, devolvemos `outputs`, los estados ocultos hacia adelante y hacia atrás para cada token en la oración origen apilados. También devolvemos `hidden`, que actúa como nuestro estado oculto inicial en el decoder.

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el encoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        self.gru= nn.GRU(self.emb_dim, self.enc_hid_dim, bidirectional = True)
        self.linear= nn.Linear(self.enc_hid_dim*2,self.dec_hid_dim )
        self.tanh = nn.Tanh()
        self.dropout_ = nn.Dropout(self.dropout)

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, src):
        ########################################################################
        # TODO: Implementá la función forward para el encoder. Deberías        #
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        
        #src = [src len, batch size]
        
        embedded = self.embedding(src)
        embedeed = self.dropout_(embedded)
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.gru(embedded)
             
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden tiene las variables ocultas apiladas [forward_1, backward_1, forward_2, backward_2, ...]
        #los outputs provienen siempre de la última capa
        
        #hidden [-2, :, : ] es la última capa de las RNN hacia adelante
        #hidden [-1, :, : ] es la última capa de las RNN hacia atrás
        
        #el estado oculto inicial del decoder es el estado oculto final del encoder
        #tanto de las capas hacia adelante como hacia atrás que se pasan por una capa densa

        concatenado = torch.cat((hidden [-2, :, : ], hidden [-1, :, : ]), 1 )
        salida_lineal = self.linear(concatenado)
        hidden = self.tanh(salida_lineal)

        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        return outputs, hidden

### Atención

Lo siguiente es la capa de atención. Esto tomará el estado oculto anterior del decoder, $ s_{t-1} $, y todos los estados ocultos hacia adelante y hacia atrás del encoder apilados, $ H $. La capa generará un vector de atención, $ a_t $, que tiene la misma longitud de la oración origen, donde cada elemento está entre 0 y 1 y el vector completo suma 1.

Intuitivamente, esta capa toma lo que hemos decodificado hasta ahora, $ s_{t-1} $, y todo lo que hemos codificado, $ H $, para producir un vector, $ a_t $, que representa a qué palabras en la oración origen debemos prestar la mayor atención para predecir correctamente la siguiente palabra a decodificar, $ \hat {y}_{t + 1} $.

Primero, calculamos la *energía* entre el estado oculto anterior del decoder y los estados ocultos del encoder. Como los estados ocultos de nuestro encoder son una secuencia de $ T $ tensores , y nuestro estado oculto anterior del decoder es un solo tensor, lo primero que hacemos es `repetir` el estado oculto anterior del decoder $ T $ veces. Luego calculamos la energía, $ E_t $, entre ellos concateándolos y pasándolos a través de una capa lineal (`attn`) y una función de activación $ \tanh $. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

Esto se puede considerar como un cálculo de qué tan bien cada estado oculto del encoder "coincide" con el estado oculto anterior del decoder.

Actualmente tenemos un tensor **[dimensión de variables ocultas del decoder, longitud de oración de origen]** para cada ejemplo del lote. Queremos que sea **[longitud de oración de origen]** para cada ejemplo en el lote, ya que la atención debe estar sobre la longitud de la oración de origen. Esto se logra multiplicando la `energía` por un tensor **[1, dimensión de variables ocultas del decoder]**, $ v $.

$$\hat{a}_t = v E_t$$

Podemos pensar en $ v $ como los pesos de una suma ponderada de la energía en todos los estados ocultos del encoder. Estos pesos nos dicen cuánto debemos prestar atención a cada token en la secuencia de origen. Los parámetros de $ v $ se inicializan aleatoriamente, pero se aprenden con el resto del modelo mediante backpropagation. Tenga en cuenta que $ v $ no depende del tiempo y que se usa el mismo $ v $ para cada paso de tiempo de la decodificación. Implementamos $ v $ como una capa lineal sin sesgo.

Finalmente, nos aseguramos de que el vector de atención se ajuste a las restricciones de tener todos los elementos entre 0 y 1 y el vector sumando 1 pasándolo a través de una capa $ \text{softmax} $.

$$a_t = \text{softmax}(\hat{a_t})$$

¡Esto nos da la atención sobre la oración original!

Gráficamente, esto se parece a lo siguiente. Esto es para calcular el primer vector de atención, donde $ s_{t-1} = s_0 = z $. Los bloques verde/verde azulado representan los estados ocultos de los RNN hacia adelante y hacia atrás, y el cálculo de atención se realiza dentro del bloque rosa.

![](https://i.imgur.com/epVe8kC.png)

In [16]:
import torch.nn.functional as F

In [29]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        #########################################################################
        #TO_DO: Configure las capas que necesita para la capa de atención con la#
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        super(Attention, self).__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.linear = nn.Linear(enc_hid_dim*2+dec_hid_dim,dec_hid_dim)
        self.tanh = nn.Tanh() 
        
          
    def forward(self, hidden, encoder_outputs):
        ########################################################################
        # TODO: Implementá la función forward para la capa de atención. Deberías
        # usar las capas que definiste en __init__ y especificar la            #
        # conectividad de dichas capas                                         #
        ########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        src_len = encoder_outputs.shape[0]
        
        
        #repetí el estado oculto del decoder src_len veces
        hidden = hidden.unsqueeze(1).repeat(1,src_len,1)
        hidden = torch.transpose(hidden,0,1)
        concat = torch.cat((hidden, encoder_outputs), 2)

      
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        linear = self.linear(concat)
        energy = self.tanh(linear)
        
        energy = torch.transpose(energy,0,1)
        
        #energy = [batch size, src len, dec hid dim]
        

        v = torch.rand((self.dec_hid_dim), requires_grad=True).to(device)
        
        
        attention = torch.matmul(energy,v)

        
        
        #attention= [batch size, src len]

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        return F.softmax(attention, dim=1)

### Decoder

Lo siguiente es el decoder.

El decoder contiene la capa de atención, `attention`, que toma el estado oculto anterior, $ s_{t-1} $, todos los estados ocultos del encoder, $ H $, y devuelve el vector de atención, $ a_t $.

Luego usamos este vector de atención para crear un vector origen pesado, $ w_t $, denotado por `weighted`, que es una suma ponderada de los estados ocultos del encoder, $ H $, usando $ a_t $ como pesos.

$$w_t = a_t H$$

El embedding de la palabra de entrada, $ d (y_t) $, el vector origen pesado, $ w_t $, y el estado oculto anterior del decoder, $ s_ {t-1} $, se pasan al decoder RNN, con $ d( y_t) $ y $ w_t $ concatenados.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

Luego pasamos $ d (y_t) $, $ w_t $ y $ s_t $ a través de la capa lineal, $ f $, para hacer una predicción de la siguiente palabra en la oración objetivo, $ \hat{y}_{t + 1}$. Esto se hace concatenándolos todos juntos.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

La siguiente imagen muestra la decodificación de la primera palabra en una traducción de ejemplo.

![](https://i.imgur.com/bqJ2DU1.png)

Los bloques verde/verde azulado muestran los RNN del encoder hacia adelante/atrás que generan $ H $, el bloque rojo muestra el vector de contexto, $ z = h_T = \tanh (g (h ^ \rightarrow_T, h ^ \leftarrow_T)) = \tanh(g (z ^ \rightarrow, z ^ \leftarrow)) = s_0 $, el bloque azul muestra el decoder RNN que genera $ s_t $, el bloque púrpura muestra la capa lineal, $ f $, que genera $ \hat{y}_{t + 1} $ y el bloque naranja muestra el cálculo de la suma pesada sobre $ H $ por $ a_t $ y genera $ w_t $. No se muestra el cálculo de $ a_t $.

In [300]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el decoder con la         #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self. attention = attention

        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        self.gru= nn.GRU((self.enc_hid_dim * 2) + self.emb_dim, self.dec_hid_dim)
        self.linear= nn.Linear(self.enc_hid_dim*2+self.dec_hid_dim + self.emb_dim,self.output_dim )
        self.dropout_ = nn.Dropout(self.dropout)

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = torch.squeeze(input,0)
        
        #input = [1, batch size]
        
        embedded = self.embedding(input)
        embedded = torch.unsqueeze(embedded ,0)
        
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = torch.unsqueeze(a,1)

        
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = torch.transpose(encoder_outputs,0,1)

        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        
        weighted = torch.matmul(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = torch.transpose(weighted,0,1)


        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted),2 )
        hidden = torch.unsqueeze(hidden, 0)

        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.gru(rnn_input, hidden)
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers y n directions valdrá siempre 1 en el decoder, por lo tanto:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #esto tmbién significa que output == hidden
        assert (output == hidden).all()
        
        output = torch.squeeze(output,0)
        hidden = torch.squeeze(hidden,0)
        weighted = torch.squeeze(weighted,0)
        embedded = torch.squeeze(embedded,0)


        concatenado_final = torch.cat((embedded, weighted, output), 1)
        prediction = self.linear(concatenado_final)

        
        #prediction = [batch size, output dim]

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

Este es el primer modelo en el que no es necesario que el encoder RNN y el decoder RNN tengan las mismas dimensiones ocultas, sin embargo, el encoder tiene que ser bidireccional. Este requisito puede eliminarse cambiando todas las apariciones de `enc_dim * 2` a` enc_dim * 2 si encoder_is_bidirectional else enc_dim`.

Este encapsulador seq2seq es similar a los dos últimos. La única diferencia es que el `encoder` devuelve tanto el estado oculto final (que es el estado oculto final de los RNN del encoder hacia adelante y hacia atrás que pasan a través de una capa lineal) para ser utilizado como el estado oculto inicial para el decoder,  como también todos los estados ocultos (que son los estados ocultos hacia adelante y hacia atrás apilados uno encima del otro). También debemos asegurarnos de que `hidden` y` encoder_outputs` se pasen al decoder.

Repasando brevemente todos los pasos:
- el tensor de `outputs` se crea para contener todas las predicciones, $ \hat {Y} $
- la secuencia de origen, $ X $, se alimenta al encoder para recibir $ z $ y $ H $
- el estado oculto inicial del decoder se establece en el vector `context`, $ s_0 = z = h_T $
- usamos un lote de tokens `<sos>` como la primera `input`, $ y_1 $
- luego decodificamos dentro de un bucle:
   - insertando el token de entrada $ y_t $, estado oculto anterior, $ s_ {t-1} $, y todas las salidas del encoder, $ H $, en el decoder
   - recibiendo una predicción, $ \hat{y}_{t + 1} $, y un nuevo estado oculto, $ s_t $
   - luego decidiendo si vamos a hacer forzamiento del maestro o no, estableciendo la siguiente entrada según corresponda

In [301]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        #########################################################################
        #TO_DO: Configure las capas que necesita para el modelo seq2seq con la  #
        #       arquitectura definida anteriormente.                            #
        #########################################################################
        # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) ***** 

        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
        ########################################################################
        #                          FINAL DE TU CÓDIGO                          #       
        ########################################################################
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio es la probabilidad con la que se usa el forzamiento del maestro
        #e.g. si teacher_forcing_ratio es 0.75 usamos las etiquetas el 75% del tiempo
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor para almacenar las salidas del decoder 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs son todos los estados ocultos de la secuencia de origen, hacia adelante y hacia atrás
        #hidden son los estados ocultos finales haci aadelante y hacia atrás pasados a través de una capa densa
        encoder_outputs, hidden = self.encoder(src)

                
        #la primera entrada al decoder es el token <sos>
        input = trg[0,:]
        
        for t in range(1, trg_len):
            ########################################################################
            # TODO: Implementá el bucle de decodificación dentro de la función     #
            # forward para el modelo seq2seq. Deberías usar las capas que definiste#     
            # en __init__ y especificar la conectividad de dichas capas            #
            ########################################################################
            # ***** INICIO DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
                
            prediction, hidden = self.decoder(input,hidden,encoder_outputs)
            
            outputs[t] = prediction
            if np.random.uniform(0,1)<teacher_forcing_ratio:
              input = trg[t]
            else:
              input = torch.argmax(prediction, dim=1)


            # ***** FINAL DE TU CÓDIGO (NO BORRES NI MODIFIQUES ESTA LÍNEA) *****
            ########################################################################
            #                          FINAL DE TU CÓDIGO                          #       
            ########################################################################
        return outputs

## Entrenamiento del modelo Seq2Seq

El resto de esta sección es muy similar a la anterior.

Inicializamos nuestros parámetros, encoder, decoder y modelo seq2seq (colocándolo en la GPU si tenemos una).

In [302]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

Usamos una versión simplificada del esquema de inicialización de peso utilizado en el paper. Aquí, inicializaremos todos los sesgos a cero y todos los pesos con $ \mathcal {N} (0, 0.01) $.

In [303]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (gru): GRU(256, 512, bidirectional=True)
    (linear): Linear(in_features=1024, out_features=512, bias=True)
    (tanh): Tanh()
    (dropout_): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (linear): Linear(in_features=1536, out_features=512, bias=True)
      (tanh): Tanh()
    )
    (embedding): Embedding(5893, 256)
    (gru): GRU(1280, 512)
    (linear): Linear(in_features=1792, out_features=5893, bias=True)
    (dropout_): Dropout(p=0.5, inplace=False)
  )
)

Calculamos el número de parámetros. Obtenemos un aumento de casi un 50% en la cantidad de parámetros con respecto al último modelo.

In [304]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,518,405 trainable parameters


Creamos un optimizador

In [305]:
optimizer = optim.Adam(model.parameters())

Inicializamos la función de pérdida.


In [306]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Y creamos el loop de entrenamiento.

In [307]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...y el loop de evaluación, recordando configurar el modelo en modo `eval` y desactivar el forzamiento del maestro.

In [308]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #desactiva el forzamiento del maestro

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finalmente, definimos una función de temporización.

In [309]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Luego, entrenamos nuestro modelo, guardando los parámetros que nos den la mejor pérdida de validación.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 19s
	Train Loss: 5.033 | Train PPL: 153.402
	 Val. Loss: 4.937 |  Val. PPL: 139.330
Epoch: 02 | Time: 1m 21s
	Train Loss: 4.128 | Train PPL:  62.069
	 Val. Loss: 4.222 |  Val. PPL:  68.168
Epoch: 03 | Time: 1m 23s
	Train Loss: 3.302 | Train PPL:  27.173
	 Val. Loss: 3.637 |  Val. PPL:  37.965
Epoch: 04 | Time: 1m 24s
	Train Loss: 2.718 | Train PPL:  15.143
	 Val. Loss: 3.426 |  Val. PPL:  30.747
Epoch: 05 | Time: 1m 24s
	Train Loss: 2.311 | Train PPL:  10.082
	 Val. Loss: 3.340 |  Val. PPL:  28.232
Epoch: 06 | Time: 1m 24s
	Train Loss: 1.979 | Train PPL:   7.237
	 Val. Loss: 3.276 |  Val. PPL:  26.474
Epoch: 07 | Time: 1m 24s
	Train Loss: 1.726 | Train PPL:   5.619
	 Val. Loss: 3.290 |  Val. PPL:  26.830
Epoch: 08 | Time: 1m 25s
	Train Loss: 1.522 | Train PPL:   4.581
	 Val. Loss: 3.410 |  Val. PPL:  30.253
Epoch: 09 | Time: 1m 24s
	Train Loss: 1.333 | Train PPL:   3.793
	 Val. Loss: 3.432 |  Val. PPL:  30.933
Epoch: 09 | Time: 1m 24s
	Train Loss: 1.333 | Train PPL

Finalmente, probamos el modelo en el conjunto de prueba usando estos "mejores" parámetros.

In [313]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.286 | Test PPL:  26.740 |


Hemos mejorado el modelo anterior, pero esto tuvo el costo de duplicar el tiempo de entrenamiento.